<a href="https://colab.research.google.com/github/jsekiewicz/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train,y_train)),
      callbacks=[tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm,y_pred)


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size =(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size =(3,3), activation='relu', padding = 'same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding = 'same'),
    Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),    
    
    Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding = 'same'),
    Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),    

    Flatten(),

    Dense(1024, activation = 'relu'),
    Dropout(0.3),

    Dense(1024, activation = 'relu'),
    Dropout(0.3),

    Dense(num_classes, activation = 'softmax')                    
  ])


In [23]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 46ms/step - loss: 2.8662 - accuracy: 0.2399 - val_loss: 1.5735 - val_accuracy: 0.4563
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 1.3604 - accuracy: 0.5502 - val_loss: 0.5758 - val_accuracy: 0.8128
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.6906 - accuracy: 0.7717 - val_loss: 0.2033 - val_accuracy: 0.9401
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4257 - accuracy: 0.8635 - val_loss: 0.1422 - val_accuracy: 0.9537
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2962 - accuracy: 0.9059 - val_loss: 0.0584 - val_accuracy: 0.9831


0.9643990929705215

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size =(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size =(3,3), activation='relu', padding = 'same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding = 'same'),
    Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),    
    
    Conv2D(filters=128, kernel_size =(3,3), activation='relu', padding = 'same'),
    Conv2D(filters=128, kernel_size =(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),    

    Flatten(),

    Dense(1024, activation = 'relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation = 'relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation = 'softmax')                    
  ])


In [0]:
def func_obj(params): 
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit( 
      X_train, 
      y_train, 
      batch_size=int(params.get('batch_size', 128)), 
      epochs=5, 
      verbose=0 
      ) 
  score = model.evaluate(X_test,y_test,verbose=0) 
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [32]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.39254324457333445, 'dropout_cnn_block_three': 0.39639616348456885, 'dropout_cnn_block_two': 0.4055659704491641, 'dropout_dense_block_one': 0.3108306313114391, 'dropout_dense_block_two': 0.42630367774598343}
accuracy=0.9532879590988159
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3441665320437818, 'dropout_cnn_block_three': 0.46664992741404776, 'dropout_cnn_block_two': 0.4531011076879158, 'dropout_dense_block_one': 0.4837208096194092, 'dropout_dense_block_two': 0.653859051426912}
accuracy=0.8285714387893677
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.45335551877145447, 'dropout_cnn_block_three': 0.40984492858657595, 'dropout_cnn_block_two': 0.4254940858059959, 'dropout_dense_block_one': 0.41647939312391785, 'dropout_dense_block_two': 0.432955795025468}
accuracy=0.9344671368598938
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.43827999510011717, 'dropout_cnn_block_three': 0.38831912046271244, 'dropout_cnn_block_two': 0.330317735